In [20]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from keras.utils import to_categorical

# Load Dataset

In [55]:
def load_dataset(directory):
    X, y = [], []
    columns = ['Pelvis Accel Sensor X,mG', 'Pelvis Accel Sensor Y,mG',
       'Pelvis Accel Sensor Z,mG', 'Pelvis Rot X,', 'Pelvis Rot Y,',    
       'Pelvis Rot Z,']
    # Iterate through each subdirectory (falls and nonFalls)
    for subdir in os.listdir(directory):
        # Ignore any files in the directory
        if not os.path.isdir(os.path.join(directory, subdir)):
            continue
        # Iterate through each file in the subdirectory
        for file in os.listdir(os.path.join(directory, subdir)):
        
            filepath = os.path.join(directory, subdir, file)
            # Load the data from each CSV file
            data = pd.read_csv(filepath,usecols=columns)
            # Append data and labels to X and y lists
            X.append(data.values)  # Assuming all columns are features
            # Assign label based on subdirectory
            y.append(1 if subdir == 'fall' else 0)  # Assuming 'falls' is class 1, and 'nonFalls' is class 0
    return np.array(X), np.array(y)




Train and Evaluate

In [76]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy,class_weights):
    verbose, epochs, batch_size = 1, 4, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]


    n_steps, n_length = 4, 75
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))


    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=256, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=256, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.3)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(128))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','recall','precision'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose,class_weight = class_weights)
    preds = model.predict(testX, batch_size=batch_size, verbose=1)
    return np.argmax(preds,axis=1)

 

In [77]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight


# Load training data
trainX, trainy = load_dataset('./3S/train')

class_weights = compute_class_weight(class_weight = "balanced",
                                    classes = np.unique(trainy),
                                    y = trainy)

class_weights = dict(enumerate(class_weights))


trainy = to_categorical(trainy)


# Load testing data
testX, testy = load_dataset('./3S/test')
testy = to_categorical(testy)

trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 6))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 6))


trainX = np.delete(trainX, 1, axis=1)
testX =  np.delete(testX, 1, axis=1)


true_labels = [np.argmax(label) for label in testy]
preds = evaluate_model(trainX, trainy, testX, testy,class_weights)

print("Accuracy: ",  accuracy_score(true_labels, preds))
print("Precision: ", precision_score(true_labels, preds))
print("Recall: ", recall_score(true_labels, preds))
print("F1: ", f1_score(true_labels, preds))
print(confusion_matrix(true_labels,preds))




Epoch 1/4


c:\Users\mjhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 12s 186ms/step - accuracy: 0.4974 - loss: 0.6988 - precision: 0.5037 - recall: 0.5044
Epoch 2/4
47/47 ━━━━━━━━━━━━━━━━━━━━ 9s 194ms/step - accuracy: 0.7918 - loss: 0.5245 - precision: 0.7973 - recall: 0.7893
Epoch 3/4
47/47 ━━━━━━━━━━━━━━━━━━━━ 10s 204ms/step - accuracy: 0.8288 - loss: 0.4877 - precision: 0.8249 - recall: 0.8313
Epoch 4/4
47/47 ━━━━━━━━━━━━━━━━━━━━ 10s 211ms/step - accuracy: 0.8688 - loss: 0.4834 - precision: 0.8658 - recall: 0.8681
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step
Accuracy:  0.8994413407821229
Precision:  0.7058823529411765
Recall:  0.75
F1:  0.7272727272727273
[[274  20]
 [ 16  48]]
